In [2]:
import requests
import pandas as pd
import datetime as dt

In [55]:
matches = pd.read_csv('matches.csv')

In [4]:
#Sends request to get past matches

def pastMatchesRequest(date_from, date_to, competition_id, page):
    url = "https://live-score-api.p.rapidapi.com/scores/history.json"

    headers = {
        'x-rapidapi-key': "ce0be8d538msh30ecb1b4c0fca98p105373jsn1e40f1d8d1ec",
        'x-rapidapi-host': "live-score-api.p.rapidapi.com",
        }

    params = {
        'key': "MC8uSjXafFUAGJqM",
        'secret': "xRuVIJwAPNVMqsYc46HXqF7Fk2f1MGNj",
        'from': date_from,
        'to': date_to,
        'competition_id': competition_id,
        'page': page
    }

    return requests.request("GET", url, params = params,headers=headers).json()

In [5]:
#Sends request to get stats from a specific match

def matchStatsRequest(match_id):
    url = "https://live-score-api.p.rapidapi.com/matches/stats.json"

    headers = {
        'x-rapidapi-key': "ce0be8d538msh30ecb1b4c0fca98p105373jsn1e40f1d8d1ec",
        'x-rapidapi-host': "live-score-api.p.rapidapi.com",
        }

    params = {
        'key': "MC8uSjXafFUAGJqM",
        'secret': "xRuVIJwAPNVMqsYc46HXqF7Fk2f1MGNj",
        'match_id': match_id
    }

    return requests.request("GET", url, params = params,headers=headers).json()

In [69]:
#Stores information from 'Past matches' on a list

def addMatchInfo(response, country, league_name):
    for info in response['data']['match']:
        if(info['ht_score'] == ''):
            info['ht_score'] = '0 - 0'
        matches_list.append([
            info['id'],
            info['home_id'],
            info['home_name'],
            info['away_id'],
            info['away_name'],
            info['score'].split(' -')[0],
            info['score'].split('- ')[1],
            info['ht_score'].split(' -')[0],
            info['ht_score'].split('- ')[1],
            info['location'],
            'City',
            country,
            'Europe',
            league_name,
            'League',
            '1',
            info['scheduled'].split(':')[0],
            info['scheduled'].split(':')[1],
            info['date'].split('-')[2],
            dt.date(int(info['date'].split('-')[0]),int(info['date'].split('-')[1]),int(info['date'].split('-')[2])).strftime("%A"),
            info['date'].split('-')[1],
            info['date'].split('-')[0],
            'F'
        ])

In [7]:
#Updates df with 'Match stats' information

def addMatchStats(rowIndex, matchId):
    response = matchStatsRequest(matchId)
    if(response['data'] != []):
        matches.at[rowIndex,'home_yc'] = response['data']['yellow_cards'].split(':')[0]
        matches.at[rowIndex,'away_yc'] = response['data']['yellow_cards'].split(':')[1]
        matches.at[rowIndex,'home_rc'] = response['data']['red_cards'].split(':')[0]
        matches.at[rowIndex,'away_rc'] = response['data']['red_cards'].split(':')[1]
        matches.at[rowIndex,'home_possession'] = response['data']['possesion'].split(':')[0]
        matches.at[rowIndex,'away_possession'] = response['data']['possesion'].split(':')[1]
        matches.at[rowIndex,'home_shots'] = response['data']['attempts_on_goal'].split(':')[0] 
        matches.at[rowIndex,'away_shots'] = response['data']['attempts_on_goal'].split(':')[1] 
        if(response['data']['shots_on_target'] is None):
            matches.at[rowIndex,'home_shots_on_target'] = int(response['data']['attempts_on_goal'].split(':')[0]) - int(response['data']['shots_off_target'].split(':')[0])
            matches.at[rowIndex,'away_shots_on_target'] = int(response['data']['attempts_on_goal'].split(':')[1]) - int(response['data']['shots_off_target'].split(':')[1])
        else:
            matches.at[rowIndex,'home_shots_on_target'] = response['data']['shots_on_target'].split(':')[0]
            matches.at[rowIndex,'away_shots_on_target'] = response['data']['shots_on_target'].split(':')[1]
        matches.at[rowIndex,'home_fouls'] = response['data']['fauls'].split(':')[0]
        matches.at[rowIndex,'away_fouls'] = response['data']['fauls'].split(':')[1]
        matches.at[rowIndex,'home_attacks'] = response['data']['attacks'].split(':')[0]
        matches.at[rowIndex,'away_attacks'] = response['data']['attacks'].split(':')[1]
    matches.at[rowIndex,'got_stats'] = 'T'

In [68]:
#Gets past matches and stores it on a df

def getMatches(date_from, date_to, competition_id, country, league_name):
    page = 1
    
    response = pastMatchesRequest(date_from, date_to, competition_id, page)
    addMatchInfo(response, country, league_name)
    
    total_pages = response['data']['total_pages']
    page += 1
    
    while(page <= total_pages):
        response = pastMatchesRequest(date_from, date_to, competition_id, page)
        addMatchInfo(response, country, league_name)
        page += 1
    
    return pd.DataFrame(matches_list, columns =['match_id','home_id','home_name','away_id','away_name','home_ft_goals','away_ft_goals','home_ht_goals','away_ht_goals','stadium_name','city','country','continent','competition_name','type','tier','hour','minute','day','dayofweek','month','year','got_stats'])

In [41]:
#Gets match stats from top to bottom until an error is found

def getMatchStats():
    while(True):
        rowIndex = matches[matches['got_stats'] == 'F'].head(1).index
        matchId = matches[matches['got_stats'] == 'F'].head(1)['match_id']
        addMatchStats(rowIndex, matchId)

# Competition Ids

1 - Bundesliga

2 - Premier League

3 - La Liga

4 - Serie A

5 - Ligue 1

6 - Turkish Super Lig

7 - Russian Premier League

8 - Primeira Liga

In [84]:
matches_list = []
df = getMatches('2019-08-01','2020-08-01', 7, 'Premier League', 'Russia')
df

,match_id,home_id,home_name,away_id,away_name,home_ft_goals,away_ft_goals,home_ht_goals,away_ht_goals,stadium_name,city,country,continent,competition_name,type,tier,hour,minute,day,dayofweek,month,year,got_stats
0,151641,1955,Ural,1422,FC Rostov,2,2,1,0,Ekaterinburg Arena,City,Premier League,Europe,Russia,League,1,11,00,03,Saturday,08,2019,F
1,151721,1195,Krylya Sovetov Samara,161,FC Lokomotiv Moscow,1,2,0,1,Samara Arena,City,Premier League,Europe,Russia,League,1,13,30,03,Saturday,08,2019,F
2,151916,140,FC Spartak Moscow,1108,FC Dinamo Moscow,0,0,0,0,Otkrytiye Arena,City,Premier League,Europe,Russia,League,1,16,00,03,Saturday,08,2019,F
3,151976,178,Zenit St. Petersburg,1391,FC Krasnodar,1,1,0,0,None,City,Premier League,Europe,Russia,League,1,18,30,03,Saturday,08,2019,F
4,152164,1189,Republican FC Akhmat Grozny,1193,FC Orenburg,2,1,0,0,Akhmat Arena,City,Premier League,Europe,Russia,League,1,17,00,05,Monday,08,2019,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,201414,1422,FC Rostov,178,Zenit St. Petersburg,1,2,1,0,Rostov Arena,City,Premier League,Europe,Russia,League,1,16,00,22,Wednesday,07,2020,F
209,201415,1190,FC Ufa,694,Arsenal Tula,0,0,0,0,Stadion Neftyanik,City,Premier League,Europe,Russia,League,1,16,00,22,Wednesday,07,2020,F
210,201416,1391,FC Krasnodar,1189,FK Akhmat,4,0,1,0,Stadion FK Krasnodar,City,Premier League,Europe,Russia,League,1,16,00,22,Wednesday,07,2020,F
211,201417,695,Rubin Kazan,140,Spartak Moscow,1,2,1,1,Kazan' Arena,City,Premier League,Europe,Russia,League,1,16,00,22,Wednesday,07,2020,F


In [85]:
matches = matches.append(df, ignore_index = True)
matches.to_csv('matches.csv', index=False);
matches

,match_id,home_id,home_name,away_id,away_name,home_ft_goals,away_ft_goals,home_ht_goals,away_ht_goals,stadium_name,city,country,continent,competition_name,type,tier,hour,minute,day,dayofweek,month,year,got_stats,home_yc,away_yc,home_rc,away_rc,home_possession,away_possession,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_fouls,away_fouls,home_attacks,away_attacks
0,152587,7,Liverpool,492,Norwich City,4,1,0,0,Anfield,City,England,Europe,Premier League,League,1,19,0,9,Friday,8,2019,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,152653,14,West Ham United,12,Manchester City,0,5,0,1,NaN,City,England,Europe,Premier League,League,1,11,30,10,Saturday,8,2019,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,152730,1,AFC Bournemouth,281,Sheffield United,1,1,0,0,NaN,City,England,Europe,Premier League,League,1,14,0,10,Saturday,8,2019,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,152731,8,Burnley,4,Southampton,3,0,0,0,Turf Moor,City,England,Europe,Premier League,League,1,14,0,10,Saturday,8,2019,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,152732,3,Crystal Palace,20,Everton,0,0,0,0,NaN,City,England,Europe,Premier League,League,1,14,0,10,Saturday,8,2019,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,201414,1422,FC Rostov,178,Zenit St. Petersburg,1,2,1,0,Rostov Arena,City,Premier League,Europe,Russia,League,1,16,00,22,Wednesday,07,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2223,201415,1190,FC Ufa,694,Arsenal Tula,0,0,0,0,Stadion Neftyanik,City,Premier League,Europe,Russia,League,1,16,00,22,Wednesday,07,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2224,201416,1391,FC Krasnodar,1189,FK Akhmat,4,0,1,0,Stadion FK Krasnodar,City,Premier League,Europe,Russia,League,1,16,00,22,Wednesday,07,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2225,201417,695,Rubin Kazan,140,Spartak Moscow,1,2,1,1,Kazan' Arena,City,Premier League,Europe,Russia,League,1,16,00,22,Wednesday,07,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
#getMatchStats()
#matches.to_csv('matches.csv', index=False);

In [77]:
matches

,match_id,home_id,home_name,away_id,away_name,home_ft_goals,away_ft_goals,home_ht_goals,away_ht_goals,stadium_name,city,country,continent,competition_name,type,tier,hour,minute,day,dayofweek,month,year,got_stats,home_yc,away_yc,home_rc,away_rc,home_possession,away_possession,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_fouls,away_fouls,home_attacks,away_attacks
0,152587,7,Liverpool,492,Norwich City,4,1,0,0,Anfield,City,England,Europe,Premier League,League,1,19,0,9,Friday,8,2019,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,152653,14,West Ham United,12,Manchester City,0,5,0,1,NaN,City,England,Europe,Premier League,League,1,11,30,10,Saturday,8,2019,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,152730,1,AFC Bournemouth,281,Sheffield United,1,1,0,0,NaN,City,England,Europe,Premier League,League,1,14,0,10,Saturday,8,2019,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,152731,8,Burnley,4,Southampton,3,0,0,0,Turf Moor,City,England,Europe,Premier League,League,1,14,0,10,Saturday,8,2019,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,152732,3,Crystal Palace,20,Everton,0,0,0,0,NaN,City,England,Europe,Premier League,League,1,14,0,10,Saturday,8,2019,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,201258,800,Granada,24,Athletic Bilbao,4,0,1,0,Estadio Nuevo Los Cármenes,City,Spain,Europe,La Liga,League,1,19,00,19,Sunday,07,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1061,201259,25,Leganes,27,Real Madrid,2,2,1,1,Estadio Municipal de Butarque,City,Spain,Europe,La Liga,League,1,19,00,19,Sunday,07,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1062,201260,28,Levante,30,Getafe,1,0,0,0,Estadio Ciudad de Valencia,City,Spain,Europe,La Liga,League,1,19,00,19,Sunday,07,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1063,201261,794,Osasuna,1814,Mallorca,2,2,1,1,Estadio El Sadar,City,Spain,Europe,La Liga,League,1,19,00,19,Sunday,07,2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
